# Titanic med CatBoost

## Frågeställningar
1) Vad för typ av boosting är det?
    * "It provides a gradient boosting framework which among other features attempts to solve for Categorical features using a permutation driven alternative compared to the classical algorithm." -Wikipedia
2) Behövs encoding för kategoriska värden?
    * Nej
3) Kan den hantera missing values? (NaN/None etc)
    * Ja, det finns en parameter "nan_mode" som avgör om NaN ger fel eller ska tolkas som max eller min.
4) Hur lång tid tar det att träna en modell
5) Hur många hyperparametrar finns det och vilka är de viktigaste
6) Hur bra fungerar den på större dataset?

#### Importerar bibliotek

In [11]:
import catboost as cb
import pandas as pd
from sklearn.model_selection import train_test_split

#### Importerar data

In [12]:
titanic_data = pd.read_csv('train_titanic.csv')

In [13]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
cleaned_titanic = titanic_data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

In [15]:
mapping = {'male': 1, 'female': 0}
numeric_titanic = cleaned_titanic.replace({'Sex': mapping})

C:\Users\swedm\AppData\Local\Temp\ipykernel_20712\1787074001.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  numeric_titanic = cleaned_titanic.replace({'Sex': mapping})


In [16]:
numeric_titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [17]:
X = numeric_titanic.drop('Survived', axis = 1)
y = numeric_titanic['Survived']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [19]:
model = cb.CatBoostClassifier(train_dir=None)

In [20]:
model.fit(X_train, y_train)